In [ ]:
import pandas as pd

# Load dataset (already uploaded)
df = pd.read_csv('/challenge_train_1_percent.csv')
df = pd.read_csv('/challenge_train_2_percent.csv')
df = pd.read_csv('/challenge_train_5_percent.csv')
df = pd.read_csv('/challenge_train_10_percent.csv')
df = pd.read_csv('/challenge_train_20_percent.csv')

# Extract non-empty dialogue
texts = df['target'].dropna().tolist()

# Optional: Remove duplicates and clean up
texts = list(set(texts))
texts = [t.strip() for t in texts if len(t.strip()) > 0]

print(f"Total training samples: {len(texts)}")
print("\nSample:\n", texts[0])

Total training samples: 1021

Sample:
 When you refer to Shadow of the Tomb Raider, do you mean the one that's a third person adventure shooter?


In [ ]:
!pip install transformers datasets


In [ ]:
from transformers import GPT2Tokenizer
from datasets import Dataset

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

# Create a HuggingFace Dataset
dataset = Dataset.from_dict({"text": texts})

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 799.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Load the pre-trained model
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./distilgpt2-finetuned-viggo",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=1,
    logging_steps=10,
    fp16=True if torch.cuda.is_available() else False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Start training
trainer.train()


/tmp/ipython-input-8-2787929204.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mandirabasak616 (mandirabasak616-narula-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.693900
20,3.332800
30,3.125100
40,3.114400
50,2.955500
60,3.186100
70,2.871200
80,2.760000
90,2.740400
100,2.669600


TrainOutput(global_step=768, training_loss=2.2151281101008258, metrics={'train_runtime': 5759.115, 'train_samples_per_second': 0.532, 'train_steps_per_second': 0.133, 'total_flos': 100043993382912.0, 'train_loss': 2.2151281101008258, 'epoch': 3.0})

In [ ]:
trainer.save_model("distilgpt2-finetuned-viggo")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2-finetuned-viggo")
output = generator("Tell me about the game", max_length=50)
print(output[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Tell me about the game, from the first person perspective, that you didn't like it? Is it a multiplayer shooter that has no multiplayer? I really enjoy playing it. Would you say that you don't like multiplayer games on PlayStation? My favorite is The Witcher 3: Wild Hunt. Do you like third person action-adventure games on Steam? I do. Do you enjoy third person action-adventure games on Linux or Mac? I have no preference for Linux or Mac. Do you prefer a Linux or a Mac release? I don't like Linux or Mac releases. Have you played any third person game with no multiplayer? I like playing single-player games on Linux or Mac. Have you played any third person game that does not have multiplayer? I don't like playing single-player games on Linux or Mac. Do you play multiplayer games on Mac? I like playing single-player games on Linux or Mac. Do you enjoy playing single-player games on Linux or Mac? I like playing single-player games on Linux or Mac. Do you enjoy playing single-player games on

In [ ]:
prompts = [
    "Tell me about the game",
    "Do you know anything about The Witcher 3?",
    "What kind of game is Portal 2?",
    "Is Skyrim a good game?",
]

for prompt in prompts:
    output = generator(prompt, max_length=50, do_sample=True, top_k=50)
    print(f"\nPrompt: {prompt}")
    print("Generated:", output[0]['generated_text'])


Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: Tell me about the game
Generated: Tell me about the game I played? Do you find it to be a real good game? I usually have no opinion, but I find it to be an average rating. Do you have a Linux or Mac release? Is it usually a poor choice for most games? Do you have a Linux or Mac release? Do you have a Mac release? Do you have a Mac release? Do you have a Linux or Mac release? Do you have a Mac release? Do you have a Mac release? If you know of any Linux games, please let me know! Are you a fan of PC games? I love indie adventure games like The Forest of Doom. Have you played any of them? Do you have a Linux or Mac release? Do you have a Mac release? Have you played any of them? It's a good game. Have you played any of them? It was released in 2013. Do you usually play Linux games? I'm usually not a fan of Linux games, but I find it to be a good game. The fact that you have a Linux or Mac release makes me especially curious. Do you usually play games on PC? Do you normally play 

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: Do you know anything about The Witcher 3?
Generated: Do you know anything about The Witcher 3? Have you heard of it yet? Have you played it before? Have you played any of it before? Do you think it's okay for your PC, Mac or Linux users? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it before? Have you played it befor

Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: What kind of game is Portal 2?
Generated: What kind of game is Portal 2? It's pretty awesome. It's a game on the PC, but that's not the case for games on Steam, as it's not supported on Linux. It's not on Mac. It's not on Linux. It's not on Linux or Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Mac. It's not on Linux. It's not on Mac. It's not on Mac. It's not on Mac.

Prompt: Is Skyrim a good game?
Generated: Is Skyrim a good game? Have you tried it before? Have you tried it before? Have you tried it before? 

In [1]:
!pip install gradio

import gradio as gr

def generate_game_dialogue(prompt):
    result = generator(prompt, max_length=50, do_sample=True, top_k=50)[0]["generated_text"]
    return result

gr.Interface(fn=generate_game_dialogue, inputs="text", outputs="text", title=" Game Dialogue Generator").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ece4d2c96508126426.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
